In [ ]:
!pip install xmltodict -q
import requests, csv, datetime, json, xmltodict

usgs_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson"
api_key = "8ee3c868025544bdbc1f368e1035910e"

# function for unix time conversion
def convert_unix_time(unix_time):
    orig_time_sec = unix_time / 1000
    datetime_timestamp = datetime.datetime.utcfromtimestamp(orig_time_sec)
    datetime_adj_timestamp = datetime_timestamp - datetime.timedelta(hours=7)
    time_str = datetime_adj_timestamp.strftime("%B %d, %Y at %I:%M:%S %p")
    return time_str

# function for usgs earthquake information
def earthquake_information():
    try:
        response = requests.get(usgs_url)
        response.raise_for_status()
        data = json.loads(response.text)
        earthquakes = data["features"]
        return earthquakes
    except Exception as e:
        print(f"Sorry, error: {e}")
        return []

# function for opencage location information
def location_information(lat, lng):
    try:
        url = (f"https://api.opencagedata.com/geocode/v1/xml?q={lat}+{lng}&key={api_key}")
        response = requests.get(url)
        response.raise_for_status()
        data = xmltodict.parse(response.text)
        if 'results' in data and 'components' in data['results']:
            components = data['results']['components']
            location_info = {
                "county": components.get('county', 'N/A'),
                "state": components.get('state', 'N/A')
            }
            return location_info
    except Exception as e:
        if response.status_code == 400:
            return {"county": 'N/A', "state": 'N/A'}
        else:
            print(f"Sorry, error: {e}")

def loop():
    earthquakes = earthquake_information()
    if earthquakes:
        with open("earthquakes.csv", "w", newline="") as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(["Time", "Magnitude", "Latitude", "Longitude", "County", "State"])
            for earthquake in earthquakes:
                properties = earthquake["properties"]
                geometry = earthquake["geometry"]
                magnitude = properties["mag"]
                unix_time = properties["time"]
                lat, lng = geometry["coordinates"][:2]
                time_str = convert_unix_time(unix_time)
                location_info = location_information(lat, lng)
                county = location_info["county"]
                state = location_info["state"]
                if county == 'N/A' and state == 'N/A':
                    print(f"Magnitude {magnitude} earthquake on {time_str} and located at ({lat}, {lng}). Further location detail could not be determined.")
                else:
                    print(f"Magnitude {magnitude} earthquake on {time_str} and located at ({lat}, {lng}) in {county}, {state}.")
                csv_writer.writerow([time_str, magnitude, lat, lng, county, state])
loop()

Magnitude 1.4 earthquake on October 11, 2023 at 10:41:07 AM and located at (-149.9943, 65.4655). Further location detail could not be determined.
Magnitude 1.15 earthquake on October 11, 2023 at 10:35:58 AM and located at (-118.4866638, 35.0149994). Further location detail could not be determined.
Magnitude 1.76 earthquake on October 11, 2023 at 10:26:03 AM and located at (-115.6094971, 33.0465012). Further location detail could not be determined.
Magnitude 1.41 earthquake on October 11, 2023 at 10:19:12 AM and located at (-118.1016693, 34.095665). Further location detail could not be determined.
Magnitude 1.1 earthquake on October 11, 2023 at 10:18:15 AM and located at (-116.7386, 37.0384). Further location detail could not be determined.
Magnitude 1.4 earthquake on October 11, 2023 at 10:16:38 AM and located at (-148.8022, 64.837). Further location detail could not be determined.
Magnitude 1.96 earthquake on October 11, 2023 at 10:00:34 AM and located at (-155.286666870117, 19.362167